# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** I based my architecture on the architecture in the google "Show and Tell" [paper](https://arxiv.org/pdf/1411.4555.pdf) , with the exception that the encoder is the default ResNet50. I experimented with adding batch normalization on the output but I had other problems so I didn't notice a difference. 

For the encoder I used an embedding layer, a single LSTM layer, dropout and an output Linear fully connected layer. I tried adding softmax to the output, but it didn't seem to make a difference. I also initialized the embeddings and hidden LSTM values, so I get reasonable results even after the 1. epoch. Embedding and Hidden layer size were set to 512, as this is what is used in the "Show and tell" [paper](https://arxiv.org/pdf/1411.4555.pdf).

Batch_size was set left at default 10, reducing it to 1 slows down the training drastically, and I had some memory leak when trying batch_size=64 and it didn't seem faster (and had a massive memory leak). But these experiments were done while I still had bugs in my model.py, so now things would probably look different.   

I left vocab_treshold and num_epochs at the recommended values.  

### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** The default looks reasonable, although I have some doubts as to whether it makes sense to discard the data around the edge in this case, but I didn't investigate that further.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** I stayed with the defaults. This is a good choice because while the LSTM is in the initial phases of training it could mess up the CNN coefficients of the pre-trained encoder (one of the papers I read also mentioned this would happen). It also intuitively makes sense to only train the new uninitialized layers. Maybe a performance boost could be obtained if after training the network like this some fine-tuning was done on the CNN encoder. 

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** I had a serious bug in my model optimization which caused SGD to not train it very quickly. I tried Adam after that, but for some reason (maybe too low dropout and bugs) it would always output the same sentence irrespective of the input data. So I went back to SGD with a higher learning rate and this succesfully trained the network with ok performance. 

In [13]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

import nltk
nltk.download('punkt')

## TODO #1: Select appropriate values for the Python variables below.
#batch_size=1
batch_size=10
#batch_size = 64          # batch size, we have 10GB of GPU memory, let's use it
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size, max_batch_size=batch_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters()) + list(encoder.bn.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.SGD(params, lr=0.1)

# this data is probably pretty sparse, and defaults are probably ok
#http://ruder.io/optimizing-gradient-descent/
#optimizer = torch.optim.Adam(params, lr=0.001)
#optimizer = torch.optim.Adam(params)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

[nltk_data] Downloading package punkt to /home/sthenc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...


  0%|          | 1851/414113 [00:00<00:45, 9075.73it/s]

Done (t=0.52s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:42<00:00, 9727.73it/s] 


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [14]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# temporary
%load_ext autoreload
%autoreload 2

from model import EncoderCNN, DecoderRNN

# Open the training log file.
f = open(log_file, 'w')

## running the training locally
#old_time = time.time()
#response = requests.request("GET", 
#                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
#                            headers={"Metadata-Flavor":"Google"})


for epoch in range(1, num_epochs+1):

    for i_step in range(1, total_step+1):
        
        ## running the training locally
        #if time.time() - old_time > 60:
        #    old_time = time.time()
        #    requests.request("POST", 
        #                     "https://nebula.udacity.com/api/v1/remote/keep-alive", 
        #                     headers={'Authorization': "STAR " + response.text})

        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler

        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)

        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()

        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)

        # Calculate the batch loss.
        loss = criterion(outputs.contiguous().view(-1, vocab_size), captions.contiguous().view(-1))

        # Backward pass.
        loss.backward(retain_graph=True)

        # Update the parameters in the optimizer.
        optimizer.step()

        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))

        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()

        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()

        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)

    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Epoch [1/3], Step [100/41412], Loss: 5.7521, Perplexity: 314.8569
Epoch [1/3], Step [200/41412], Loss: 6.2931, Perplexity: 540.8357
Epoch [1/3], Step [300/41412], Loss: 5.6379, Perplexity: 280.86786
Epoch [1/3], Step [400/41412], Loss: 5.6829, Perplexity: 293.8073
Epoch [1/3], Step [500/41412], Loss: 5.0780, Perplexity: 160.4558
Epoch [1/3], Step [600/41412], Loss: 5.3786, Perplexity: 216.7153
Epoch [1/3], Step [700/41412], Loss: 4.8912, Perplexity: 133.1118
Epoch [1/3], Step [800/41412], Loss: 4.8111, Perplexity: 122.8716
Epoch [1/3], Step [900/41412], Loss: 5.4779, Perplexity: 239.3480
Epoch [1/3], Step [1000/41412], Loss: 5.0323, Perplexity: 153.2923
Epoch [1/3], Step [1100/41412], Loss: 4.6186, Perplexity: 101.3516
Epoch [1/3], Step [1200/41412], Loss: 4.5781, Perplexity: 97.32837
Epoch [1/3], Step [1300/41412], Loss: 4.5420, Perplexity: 93.87784
Epoch [1/3], Step [1400/41412], Loss: 4.7758, Per

Epoch [1/3], Step [24200/41412], Loss: 3.7556, Perplexity: 42.7611
Epoch [1/3], Step [24300/41412], Loss: 2.3806, Perplexity: 10.8110
Epoch [1/3], Step [24400/41412], Loss: 3.3599, Perplexity: 28.78547
Epoch [1/3], Step [24500/41412], Loss: 3.5174, Perplexity: 33.69757
Epoch [1/3], Step [24600/41412], Loss: 3.0778, Perplexity: 21.7097
Epoch [1/3], Step [24700/41412], Loss: 3.3670, Perplexity: 28.9926
Epoch [1/3], Step [24800/41412], Loss: 3.2877, Perplexity: 26.7815
Epoch [1/3], Step [24900/41412], Loss: 3.3052, Perplexity: 27.25334
Epoch [1/3], Step [25000/41412], Loss: 3.4461, Perplexity: 31.37779
Epoch [1/3], Step [25100/41412], Loss: 3.1457, Perplexity: 23.2357
Epoch [1/3], Step [25200/41412], Loss: 3.9032, Perplexity: 49.5624
Epoch [1/3], Step [25300/41412], Loss: 2.4940, Perplexity: 12.11028
Epoch [1/3], Step [25400/41412], Loss: 3.3570, Perplexity: 28.70153
Epoch [1/3], Step [25500/41412], Loss: 3.3195, Perplexity: 27.6466
Epoch [1/3], Step [25600/41412], Loss: 2.5289, Perplexit

Epoch [2/3], Step [7200/41412], Loss: 2.8669, Perplexity: 17.58259
Epoch [2/3], Step [7300/41412], Loss: 3.0929, Perplexity: 22.0402
Epoch [2/3], Step [7400/41412], Loss: 3.3968, Perplexity: 29.8681
Epoch [2/3], Step [7500/41412], Loss: 3.0377, Perplexity: 20.8575
Epoch [2/3], Step [7600/41412], Loss: 3.2804, Perplexity: 26.5854
Epoch [2/3], Step [7700/41412], Loss: 2.6895, Perplexity: 14.7247
Epoch [2/3], Step [7800/41412], Loss: 3.4483, Perplexity: 31.4453
Epoch [2/3], Step [7900/41412], Loss: 2.7830, Perplexity: 16.1682
Epoch [2/3], Step [8000/41412], Loss: 2.5501, Perplexity: 12.8090
Epoch [2/3], Step [8100/41412], Loss: 2.7874, Perplexity: 16.2382
Epoch [2/3], Step [8200/41412], Loss: 2.9991, Perplexity: 20.0682
Epoch [2/3], Step [8300/41412], Loss: 3.5377, Perplexity: 34.3881
Epoch [2/3], Step [8400/41412], Loss: 3.2958, Perplexity: 27.0000
Epoch [2/3], Step [8500/41412], Loss: 3.0875, Perplexity: 21.9223
Epoch [2/3], Step [8600/41412], Loss: 2.9822, Perplexity: 19.7309
Epoch [2/

Epoch [2/3], Step [31600/41412], Loss: 2.8577, Perplexity: 17.4210
Epoch [2/3], Step [31700/41412], Loss: 2.6289, Perplexity: 13.8586
Epoch [2/3], Step [31800/41412], Loss: 2.8059, Perplexity: 16.5414
Epoch [2/3], Step [31900/41412], Loss: 2.8814, Perplexity: 17.8389
Epoch [2/3], Step [32000/41412], Loss: 3.5493, Perplexity: 34.7907
Epoch [2/3], Step [32100/41412], Loss: 2.7912, Perplexity: 16.3009
Epoch [2/3], Step [32200/41412], Loss: 2.4666, Perplexity: 11.7825
Epoch [2/3], Step [32300/41412], Loss: 2.5734, Perplexity: 13.11086
Epoch [2/3], Step [32400/41412], Loss: 2.8498, Perplexity: 17.2849
Epoch [2/3], Step [32500/41412], Loss: 3.0873, Perplexity: 21.9186
Epoch [2/3], Step [32600/41412], Loss: 2.7877, Perplexity: 16.2441
Epoch [2/3], Step [32700/41412], Loss: 3.0679, Perplexity: 21.49596
Epoch [2/3], Step [32800/41412], Loss: 2.9611, Perplexity: 19.3195
Epoch [2/3], Step [32900/41412], Loss: 2.6340, Perplexity: 13.9292
Epoch [2/3], Step [33000/41412], Loss: 3.7671, Perplexity: 4

Epoch [3/3], Step [14700/41412], Loss: 2.4601, Perplexity: 11.7063
Epoch [3/3], Step [14800/41412], Loss: 2.9063, Perplexity: 18.2882
Epoch [3/3], Step [14900/41412], Loss: 3.2531, Perplexity: 25.8703
Epoch [3/3], Step [15000/41412], Loss: 3.1927, Perplexity: 24.3544
Epoch [3/3], Step [15100/41412], Loss: 2.4703, Perplexity: 11.8261
Epoch [3/3], Step [15200/41412], Loss: 2.1224, Perplexity: 8.35159
Epoch [3/3], Step [15300/41412], Loss: 2.5045, Perplexity: 12.2380
Epoch [3/3], Step [15400/41412], Loss: 2.1379, Perplexity: 8.48186
Epoch [3/3], Step [15500/41412], Loss: 2.4397, Perplexity: 11.4698
Epoch [3/3], Step [15600/41412], Loss: 2.9859, Perplexity: 19.80513
Epoch [3/3], Step [15700/41412], Loss: 2.5928, Perplexity: 13.3676
Epoch [3/3], Step [15800/41412], Loss: 2.7637, Perplexity: 15.8586
Epoch [3/3], Step [15900/41412], Loss: 2.7654, Perplexity: 15.88567
Epoch [3/3], Step [16000/41412], Loss: 3.2054, Perplexity: 24.6650
Epoch [3/3], Step [16100/41412], Loss: 2.8205, Perplexity: 1

Epoch [3/3], Step [39100/41412], Loss: 2.0651, Perplexity: 7.88630
Epoch [3/3], Step [39200/41412], Loss: 3.3627, Perplexity: 28.8659
Epoch [3/3], Step [39300/41412], Loss: 3.1308, Perplexity: 22.8920
Epoch [3/3], Step [39400/41412], Loss: 2.4762, Perplexity: 11.8959
Epoch [3/3], Step [39500/41412], Loss: 3.8608, Perplexity: 47.5027
Epoch [3/3], Step [39600/41412], Loss: 2.1311, Perplexity: 8.42379
Epoch [3/3], Step [39700/41412], Loss: 2.5049, Perplexity: 12.2418
Epoch [3/3], Step [39800/41412], Loss: 2.6221, Perplexity: 13.7646
Epoch [3/3], Step [39900/41412], Loss: 2.1072, Perplexity: 8.22533
Epoch [3/3], Step [40000/41412], Loss: 2.7113, Perplexity: 15.0495
Epoch [3/3], Step [40100/41412], Loss: 2.8108, Perplexity: 16.6237
Epoch [3/3], Step [40200/41412], Loss: 2.3347, Perplexity: 10.3268
Epoch [3/3], Step [40300/41412], Loss: 2.4702, Perplexity: 11.8243
Epoch [3/3], Step [40400/41412], Loss: 3.1435, Perplexity: 23.1843
Epoch [3/3], Step [40500/41412], Loss: 3.0834, Perplexity: 21.

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.